In [1]:
import numpy as np
from scipy.signal import welch
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn import metrics 
from sklearn.svm import SVC
from sklearn.cross_validation import StratifiedKFold

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:

#files are separeted by frequency bands
X = np.zeros((5,32,40,32,6,1280)) # (frequency bands,person,trials,channels,time(10s))
for band in range(5):
    X[band] = np.load('X_'+str(band)+'.npy') #last index is all bands, others are in incresing order


In [13]:
acc_all = []

In [6]:
import pandas as pd
Y = np.load('Y_4classes.npy')
Y_ = Y[:,:,4].reshape((32,-1))
data = pd.DataFrame(np.swapaxes(Y[:,:,4].reshape((32,-1)),0,1))
df=data.apply(pd.value_counts)
Y_cut =(df>5).as_matrix()

In [7]:
scoring = ['precision_macro', 'recall_macro', 'f1_macro']
bands = [0,1,2,3,4] #4 is all bands
classes = [4]
channel = [[]]#[2,3,18,19,20],
channel_name = ['Frontal','all']
bands_name = ['4-8Hz','8-15Hz','15-32Hz','32-40Hz','all bands']
classes_name = ['H-Valence, H-Arousal','H-Valence, L-Arousal','L-Valence, H-Arousal','L-Valence, L-Arousal','all classes']
c_range=  [1e-2,1e-1, 1,1e1, 1e2]

In [4]:
def datagen(data_x,Y_c,c):
    pp = np.where(Y_c == True)[0]
    temp = np.zeros((len(pp),5,32,6,1280)) 
    np.random.seed(0)
    for i in range(len(pp)):
        t = np.where(data.iloc[:,pp[i]]==c)[0]
        l = np.random.choice(t,5, replace=False)
        temp[i] = data_x[pp[i],l,:,:,:]
    return temp,pp

In [14]:
score_means = list()
score_stds = list()
i=0
for ch in channel:
    for b in bands:
        for c in classes:
            ch_len = len(ch)
            ch_len = 32 if ch_len ==0 else ch_len
            
            if not ch:#all channels
                data_x = X[:,:,:,:,:]
            else:
                data_x = X[:,:,:,ch,:]

            data_x = data_x[b,:,:,:,:]#person, trials,5 bands,channels,time(10s)
            print data_x.shape
            if c >3:
                data_x,y_list = datagen(data_x,Y_cut[3],3)
                data_y = np.repeat(y_list,5) #create label
                print('classes :%s, Bands: %s %d subjects\n\n' % (classes_name[c],bands_name[b],data_x.shape[0]))
                _,data_x = welch(data_x, fs=128)
                data_x = np.swapaxes(data_x,-2,-3)
                data_x = data_x.reshape((32*5,6,ch_len*129))

            else:
                data_x,y_list = datagen(data_x,Y_cut[c],c)
                data_y = np.repeat(y_list,5) #create label
                print('\nChannel:%s, classes :%s, Bands: %s %d subjects\n' % (channel_name[i],classes_name[c],bands_name[b],data_x.shape[0]))
                _,data_x = welch(data_x, fs=128)
                data_x = np.swapaxes(data_x,-2,-3)
                data_x = data_x.reshape((data_x.shape[0]*5,6,ch_len*129))
                    
            
            np.random.seed(0) #reset random seed for each band, each class
#             randomList = [np.random.choice(5,5, replace=False) for i in range(data_x.shape[0])]
            skf = StratifiedKFold(data_y,n_folds=5, shuffle=True, random_state=None)
            mylist = np.array(list(skf))
            kArray = mylist[:,1]
            for k in range(5):
                train_index = np.concatenate([kArray[i] if i!=k else [] for i in range(5)])
                
                test_index = kArray[k]
                train_index = train_index.astype(int)
                test_index = test_index.astype(int)
               
                X_train, X_val, X_test = data_x[train_index[:-int(len(train_index)/4)]], data_x[train_index[-int(len(train_index)/4):]],data_x[test_index]
                y_train, y_val, y_test = data_y[train_index[:-int(len(train_index)/4)]], data_y[train_index[-int(len(train_index)/4):]], data_y[test_index]
                X_train = np.reshape(X_train,(-1,ch_len*129))
                X_val = np.reshape(X_val,(-1,ch_len*129))
                X_test = np.reshape(X_test,(-1,ch_len*129))
                y_train = np.repeat(y_train,6)
                y_val = np.repeat(y_val,6)
                y_test = np.repeat(y_test,6)
                acc_val=0
                score_acc=0
                for c_value in c_range:
                    clf = SVC(kernel='linear', C=c_value, random_state=0)
                    clf.fit(X_train, y_train)
                    acc_temp = clf.score(X_val,y_val)
                    if acc_temp > acc_val:
                        acc_val = acc_temp
                        score_acc = clf.score(X_test, y_test)
                        
    
                acc_all.append([b,c,k,score_acc])


                #score_means.append(acc_all)
                #score_stds_t.append(scores['test_'+s].std()/np.sqrt(10))
                #print('%s   mean: %f +- %f\n' % (s,scores['test_'+s].mean(),scores['test_'+s].std()/np.sqrt(10)))
    i =i+1
        

(32, 40, 32, 6, 1280)
(32, 5, 32, 6, 1280)
13
classes :all classes, Bands: 4-8Hz 32 subjects


(32, 40, 32, 6, 1280)
(32, 5, 32, 6, 1280)
13
classes :all classes, Bands: 8-15Hz 32 subjects


(32, 40, 32, 6, 1280)
(32, 5, 32, 6, 1280)
13
classes :all classes, Bands: 15-32Hz 32 subjects


(32, 40, 32, 6, 1280)
(32, 5, 32, 6, 1280)
13
classes :all classes, Bands: 32-40Hz 32 subjects


(32, 40, 32, 6, 1280)
(32, 5, 32, 6, 1280)
13
classes :all classes, Bands: all bands 32 subjects




In [16]:
np.save('SVM_ACC_ALL_allClassOnly_allBand',acc_all)

# Precision
Frontal

In [18]:
import pandas as pd
df = pd.DataFrame(score_means[0,:,:,0],columns=classes_name)
df['bands'] = pd.DataFrame(bands_name)
df

IndexError: too many indices for array

ALL channels

In [19]:
import pandas as pd
df = pd.DataFrame(score_means[1,:,:,0],columns=classes_name)
df['bands'] = pd.DataFrame(bands_name)
df

IndexError: too many indices for array

# Recall
frontal

In [20]:
df = pd.DataFrame(score_means[0,:,:,1],columns=classes_name)
df['bands'] = pd.DataFrame(bands_name)
df

IndexError: too many indices for array

All channels

In [21]:
df = pd.DataFrame(score_means[1,:,:,1],columns=classes_name)
df['bands'] = pd.DataFrame(bands_name)
df

IndexError: too many indices for array

# Average F1
Frontal

In [22]:
df = pd.DataFrame(score_means[0,:,:,2],columns=classes_name)
df['bands'] = pd.DataFrame(bands_name)
df

IndexError: too many indices for array

All channels

In [23]:
df = pd.DataFrame(score_means[1,:,:,2],columns=classes_name)
df['bands'] = pd.DataFrame(bands_name)
df

IndexError: too many indices for array

In [27]:
acc_all

[[0, 4, 0, 0.9791666666666666],
 [0, 4, 1, 0.9739583333333334],
 [0, 4, 2, 0.9583333333333334],
 [0, 4, 3, 0.96875],
 [0, 4, 4, 0.984375],
 [1, 4, 0, 0.984375],
 [1, 4, 1, 0.9791666666666666],
 [1, 4, 2, 0.96875],
 [1, 4, 3, 0.9739583333333334],
 [1, 4, 4, 0.984375],
 [2, 4, 0, 0.9895833333333334],
 [2, 4, 1, 0.984375],
 [2, 4, 2, 0.96875],
 [2, 4, 3, 0.984375],
 [2, 4, 4, 0.9895833333333334],
 [3, 4, 0, 0.3229166666666667],
 [3, 4, 1, 0.3854166666666667],
 [3, 4, 2, 0.3645833333333333],
 [3, 4, 3, 0.4114583333333333],
 [3, 4, 4, 0.4427083333333333],
 [4, 4, 0, 0.3177083333333333],
 [4, 4, 1, 0.3854166666666667],
 [4, 4, 2, 0.359375],
 [4, 4, 3, 0.40625],
 [4, 4, 4, 0.4427083333333333]]